In [1]:
from diffusion.utils import *
import d3rlpy
from sklearn.model_selection import train_test_split

base_agent_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/data'
base_synthetic_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/cluster_results/diffusion'

base_results_folder = '/Users/shubhankar/Developer/compositional-rl-synth-data/local_results/offline_learning'

In [ ]:
robot = 'Jaco'
obj = 'Plate'
obst = 'GoalWall'
subtask = 'PickPlace'

env_name = f"{robot}_{obj}_{obst}_{subtask}"

data_type = 'agent'

if data_type == 'synthetic':
    synthetic_run_id = 20
    mode = 'train'

if data_type == 'agent':
    env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=True, ignore_done=False)
    dataset = load_single_composuite_dataset(base_path=base_agent_data_path, 
                                             dataset_type='expert', 
                                             robot=robot, obj=obj, 
                                             obst=obst, task=subtask)
    dataset, _ = remove_indicator_vectors(env.modality_dims, transitions_dataset(dataset))

if data_type == 'synthetic':
    dataset = load_single_synthetic_dataset(base_path=os.path.join(base_synthetic_data_path, synthetic_run_id, mode), 
                                            robot=robot, obj=obj, 
                                            obst=obst, task=subtask)

In [3]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=dataset['observations'],
    actions=dataset['actions'],
    rewards=dataset['rewards'],
    terminals=dataset['terminals']
)

In [4]:
sac = d3rlpy.algos.SAC(scaler='standard', action_scaler='min_max')
sac.build_with_dataset(dataset)

In [ ]:
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)
eval_env = composuite.make(robot, obj, obst, subtask, ignore_done=False)

In [ ]:
evaluate_scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
rewards = evaluate_scorer(sac)

In [ ]:
sac.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=10,
        save_interval=10,
        scorers={
            "environment": evaluate_scorer
            }
        )